In [1]:
import requests

In [2]:
response=requests.get("https://www.balldontlie.io/api/v1/games")

In [3]:
response.json()["data"]

[{'id': 47179,
  'date': '2019-01-30T00:00:00.000Z',
  'home_team': {'id': 2,
   'abbreviation': 'BOS',
   'city': 'Boston',
   'conference': 'East',
   'division': 'Atlantic',
   'full_name': 'Boston Celtics',
   'name': 'Celtics'},
  'home_team_score': 126,
  'period': 4,
  'postseason': False,
  'season': 2018,
  'status': 'Final',
  'time': ' ',
  'visitor_team': {'id': 4,
   'abbreviation': 'CHA',
   'city': 'Charlotte',
   'conference': 'East',
   'division': 'Southeast',
   'full_name': 'Charlotte Hornets',
   'name': 'Hornets'},
  'visitor_team_score': 94},
 {'id': 48751,
  'date': '2019-02-09T00:00:00.000Z',
  'home_team': {'id': 2,
   'abbreviation': 'BOS',
   'city': 'Boston',
   'conference': 'East',
   'division': 'Atlantic',
   'full_name': 'Boston Celtics',
   'name': 'Celtics'},
  'home_team_score': 112,
  'period': 4,
  'postseason': False,
  'season': 2018,
  'status': 'Final',
  'time': '     ',
  'visitor_team': {'id': 13,
   'abbreviation': 'LAC',
   'city': 'LA',


Objectives: 
1. Get the probability of win of a matches and compare againist betting company
2. After finished the first objectives, use live api to extract statistic and scarp odds from betting campany


To Do:
1. Get a small portion of data of team statistic ✅
2. Pipeline it ✅
3. Build a deep learning classification
    3.1 train/test?
4. Get the probability

Optional task:
1. download data directly from kaggle
2. use make_pipeline instead
3. I have dropped the opportent team for now and remember to add it back

In [4]:
import numpy as np

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import make_scorer

Take only the Mavericks as the home team here as Mavericks are known as rely on statistics

In [6]:
import pandas as pd
df_full=pd.read_csv("games.csv")
df_full1=df_full[df_full["HOME_TEAM_ID"]==1610612742]
df=df_full1[0:100]

1. drop NA, unneccesary columns ✅
2. check which columns to pipeline ✅
3. get the team name instead of only id ✅

See if need to add below codes:
<br> def get_team(num):
<br>     return team[team["TEAM_ID"]==num]["NICKNAME"].values[0]
<br> df["HOME_TEAM_NAME"]=np.vectorize(get_team)(df["HOME_TEAM_ID"])
<br> df["HOME_TEAM_NAME"]=np.vectorize(get_team)(df["HOME_TEAM_ID"])

In [7]:
df.dropna(axis=0,inplace=True)
df.drop(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT',
       'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home'],axis=1,inplace=True)
team=pd.read_csv("teams.csv")

X_train=df.drop(["HOME_TEAM_WINS","HOME_TEAM_ID","TEAM_ID_away"],axis=1)
y_train=df[["HOME_TEAM_WINS"]]

/tmp/ipykernel_1981/536401647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0,inplace=True)
/tmp/ipykernel_1981/536401647.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT',


In [8]:
X_train

,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away
26,77.0,0.314,0.680,0.136,11.0,37.0,1610612752,107.0,0.429,0.788,0.265,20.0,55.0
41,111.0,0.443,0.800,0.370,19.0,43.0,1610612762,103.0,0.434,0.792,0.387,22.0,45.0
53,114.0,0.476,0.741,0.389,22.0,39.0,1610612758,113.0,0.489,0.667,0.360,24.0,45.0
72,122.0,0.529,0.750,0.459,29.0,32.0,1610612744,113.0,0.543,0.769,0.500,26.0,37.0
169,97.0,0.412,0.632,0.385,16.0,44.0,1610612746,99.0,0.524,0.545,0.318,23.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2848,107.0,0.429,0.708,0.279,23.0,49.0,1610612746,110.0,0.371,0.788,0.316,19.0,57.0
2882,120.0,0.438,0.786,0.426,19.0,43.0,1610612757,112.0,0.518,0.682,0.300,25.0,40.0
2922,109.0,0.506,0.667,0.407,22.0,54.0,1610612755,91.0,0.378,0.667,0.243,22.0,43.0
2932,114.0,0.406,0.676,0.289,19.0,47.0,1610612747,129.0,0.528,0.781,0.417,18.0,49.0


Try to use make_pipeline below

In [9]:
"""
preproc_numerical_baseline = make_pipeline(StandardScaler())
preproc_categorical_baseline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))
preproc_baseline = make_column_transformer(
    (preproc_numerical_baseline, ['PTS_home','FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away']),
    (preproc_categorical_baseline, ["HOME_TEAM_ID","VISITOR_TEAM_ID"]),
    remainder="drop")
preproc_baseline
"""

'\npreproc_numerical_baseline = make_pipeline(StandardScaler())\npreproc_categorical_baseline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))\npreproc_baseline = make_column_transformer(\n    (preproc_numerical_baseline, [\'PTS_home\',\'FG_PCT_home\',\n       \'FT_PCT_home\', \'FG3_PCT_home\', \'AST_home\', \'REB_home\',\n       \'PTS_away\', \'FG_PCT_away\', \'FT_PCT_away\', \'FG3_PCT_away\', \'AST_away\',\n       \'REB_away\']),\n    (preproc_categorical_baseline, ["HOME_TEAM_ID","VISITOR_TEAM_ID"]),\n    remainder="drop")\npreproc_baseline\n'

In [13]:
"""
num_transformer = Pipeline([('scaler', StandardScaler())])
cat_transformer = OneHotEncoder(handle_unknown='ignore',sparse=False)
preprocessor = ColumnTransformer([
    ('num_tr', num_transformer, ['PTS_home','FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away']),
    ('cat_tr', cat_transformer, ["TEAM_ID_away"])],remainder='passthrough')
"""

In [21]:
X_train_scaled = StandardScaler().fit_transform(X_train)

In [22]:
y_train

,HOME_TEAM_WINS
26,0
41,1
53,1
72,1
169,0
...,...
2848,0
2882,1
2922,1
2932,0


In [16]:
X_train

,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away
26,77.0,0.314,0.680,0.136,11.0,37.0,1610612752,107.0,0.429,0.788,0.265,20.0,55.0
41,111.0,0.443,0.800,0.370,19.0,43.0,1610612762,103.0,0.434,0.792,0.387,22.0,45.0
53,114.0,0.476,0.741,0.389,22.0,39.0,1610612758,113.0,0.489,0.667,0.360,24.0,45.0
72,122.0,0.529,0.750,0.459,29.0,32.0,1610612744,113.0,0.543,0.769,0.500,26.0,37.0
169,97.0,0.412,0.632,0.385,16.0,44.0,1610612746,99.0,0.524,0.545,0.318,23.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2848,107.0,0.429,0.708,0.279,23.0,49.0,1610612746,110.0,0.371,0.788,0.316,19.0,57.0
2882,120.0,0.438,0.786,0.426,19.0,43.0,1610612757,112.0,0.518,0.682,0.300,25.0,40.0
2922,109.0,0.506,0.667,0.407,22.0,54.0,1610612755,91.0,0.378,0.667,0.243,22.0,43.0
2932,114.0,0.406,0.676,0.289,19.0,47.0,1610612747,129.0,0.528,0.781,0.417,18.0,49.0


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import Sequential, layers


In [ ]:
X_train_transformed.shape[-1]

In [ ]:
model =Sequential()
model.add(layers.Dense(10,activation="relu",input_dim=77))
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train1, y_train, batch_size=32, epochs=500)

In [ ]:
df=pd.read_csv("games.csv")
df.dtypes

In [ ]:
df1=df.drop(["GAME_DATE_EST","GAME_ID","GAME_STATUS_TEXT","HOME_TEAM_ID","VISITOR_TEAM_ID","SEASON","TEAM_ID_home","TEAM_ID_away"],axis=1)

In [ ]:
x_train=df1.drop("HOME_TEAM_WINS",axis=1)
y_train=df1[["HOME_TEAM_WINS"]]

In [ ]:
model =Sequential()
model.add(layers.Dense(10,activation="relu",input_dim=len(X)))
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))